In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from flapper_sim import FlapperSim

f = FlapperSim(robot_pose=np.array([0, 0, 0.8, 1.57]))
position = f.get_output_measurement()
y_1 = position[0]
y_2 = position[1]
y_3 = position[2]

In [ ]:
position_log = []
target_position_log = []
estimated_state_log = []

In [ ]:
# Discrete time state space model of robot
T = 0.1  # Sampling time

A_d = np.array([[1, 0.1, 0.005],
                [0, 1, 0.1],
                [0, 0, 1]])

B_d = np.array([[0.00016666667], [0.005], [0.1]])

C_d = np.array([[1, 0, 0]])

In [ ]:
# State feedback controller
# F = 10**3 * np.array([[-1.000000000000000, -0.2, -0.018333333333333]])
F = np.array([[-64.000000000000341, -41.600000000000023,  -9.813333333333333]])
# F = 10**2 * np.array([[-3.52, -1.948,  -0.18673333333333]])

# State observer
# L = np.array([[-3], [-25], [-100]])
L = np.array([[-1.090066116184267], [-4.528759116822354], [-7.739556556092396]])
# L = np.array([[-1.529582004419328], [-5.871028337456465], [-9.493798716337043]])

In [ ]:
# Target path
P0 = np.array([0, 0, 0.8])
# P1 = np.array([0.5, 0.5, 1.2])
P1 = np.array([0.75, 0.75, 1.2])
t0 = 0
t1 = 20

In [ ]:
# Target state function
def get_target_x(time):
    if time <= t0:
        return np.vstack(([P0], [0, 0, 0], [0, 0, 0])) # 
    elif time > t1:
        return np.vstack(([P1], [0, 0, 0], [0, 0, 0])) #
    elif time <= 0.5 * (t0 + t1):
        pos = 2 * (P1 - P0) * ((time - t0) / (t1 - t0)) ** 2 + P0
        vel = 4 * (P1 - P0) * (time - t0) / (t1 - t0) ** 2
        acc = 8 * (P1 - P0) / (t1 - t0) ** 2
        return np.vstack((pos, vel, acc))
    elif time > 0.5 * (t0 + t1):
        pos = P1 - 2 * (P1 - P0) * ((time - t1) / (t1 - t0)) ** 2
        vel = -4 * (P1 - P0) * (time - t1) / (t1 - t0) ** 2
        acc = -8 * (P1 - P0) / (t1 - t0) ** 2
        return np.vstack((pos, vel, acc))

In [ ]:
# Initial state estimates
x_hat_1 = np.array([position[0], [0], [0]])
x_hat_2 = np.array([position[1], [0], [0]])
x_hat_3 = np.array([position[2], [0], [0]])

In [ ]:
print(x_hat_1.shape)

In [ ]:
# Simulation loop
t = 0

for i in range(600):
    estimated_state_log.append(([x_hat_1[0, 0]], [x_hat_2[0, 0]], [x_hat_3[0, 0]]))
    target_x = get_target_x(t)  # Example target positions

    # print(target_x)
    # print(np.expand_dims(target_x[:, 0], axis=-1).shape)
    # raise NotImplementedError("Target x shape is not correct, expected (3, 1)")

    u_1 = F @ (x_hat_1 - np.expand_dims(target_x[:, 0], axis=-1))
    u_2 = F @ (x_hat_2 - np.expand_dims(target_x[:, 1], axis=-1))
    u_3 = F @ (x_hat_3 - np.expand_dims(target_x[:, 2], axis=-1))

    # print("u_1.shape = ", u_1.shape)

    # Set the control inputs to zero for debugging
    # u_1 = u_1# * 0.01
    # u_2 = u_2# * 0.01
    # u_3 = u_3# * 0.01

    x_hat_1 = (A_d + (L @ C_d)) @ x_hat_1 + B_d * u_1 - L * y_1
    x_hat_2 = (A_d + (L @ C_d)) @ x_hat_2 + B_d * u_2 - L * y_2
    x_hat_3 = (A_d + (L @ C_d)) @ x_hat_3 + B_d * u_3 - L * y_3

    f.step(u=np.array([u_1[0], u_2[0], u_3[0]]))

    position = f.get_output_measurement()
    # position = np.ones_like(position)

    position_log.append(position)
    target_position_log.append(([target_x[0,0]], [target_x[0,1]], [target_x[0,2]]))

    y_1 = position[0]
    y_2 = position[1]
    y_3 = position[2]

    # # Zero the measurements for debugging
    # y_1 = y_1 * 0
    # y_2 = y_2 * 0
    # y_3 = y_3 * 0

    t += T

In [ ]:
print(np.array(position_log).shape)

In [ ]:
print(target_position_log)

In [ ]:
print(np.array(estimated_state_log).shape)

In [ ]:
import plotly.graph_objects as go

# Plot the log of output measurements in 3d
fig = go.Figure()
fig = go.Figure(data=[go.Scatter3d(
    x=np.array(position_log)[:, 0, 0],
    y=np.array(position_log)[:, 1, 0],
    z=np.array(position_log)[:, 2, 0],
    mode='lines+markers',
    marker=dict(size=4),
    line=dict(width=2),
    name='Measured Position'  # Added label
)])
fig.add_trace(go.Scatter3d(
    x=np.array(target_position_log)[:, 0, 0],
    y=np.array(target_position_log)[:, 1, 0],
    z=np.array(target_position_log)[:, 2, 0],
    mode='lines+markers',
    marker=dict(size=4, color='red'),
    line=dict(width=2, color='red'),
    name='Target Position'  # Added label
))
fig.add_trace(go.Scatter3d(
    x=np.array(estimated_state_log)[:, 0, 0],
    y=np.array(estimated_state_log)[:, 1, 0],
    z=np.array(estimated_state_log)[:, 2, 0],
    mode='lines+markers',
    marker=dict(size=4, color='green'),
    line=dict(width=2, color='green'),
    name='Estimated State'  # Added label
))

fig.update_layout(
    title='Measured Position vs Target Position Over Time',
    scene=dict(
        xaxis_title='x',
        yaxis_title='y',
        zaxis_title='z',
        xaxis=dict(range=[-1.1, 2.1]),
        yaxis=dict(range=[-2.1, 2.1]),
        zaxis=dict(range=[0.3, 1.6])
    ),
    width=800,
    height=600,
    legend=dict(
        x=0.7,
        y=0.1,
        bgcolor="rgba(255, 255, 255, 0.5)"
    )
)
fig.show()